In [ ]:
!pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
import json
import openai

openai.api_key = "API_Key"

In [ ]:
# Zero-shot prompting with direct LLM Call
from openai import OpenAI

client = OpenAI(api_key=openai.api_key)

_system_message = """
You are a tourism expert. You should be respectful to the users and
deliver responses in a friendly manner. In case if you don't have confidence,
just say don't know. Output your responses in JSON format.
"""

_user_message = """ I am planning to travel Western side of Kashmir during
the Summer season by road. Provide me the list of 4 visiting places in Kashmir
in no more than 30 words.
"""
response = client.chat.completions.create(
    model="gpt-4.1",
    temperature=0.3,
    top_p=0.4,
    messages=[
        {"role":"system","content":_system_message},
        {"role":"user","content":_user_message}
    ]
)

print(response.choices[0].message.content)

{
  "places": [
    "Gulmarg – Famous for its lush meadows and cable car rides.",
    "Pahalgam – Known for scenic valleys and Lidder River.",
    "Sonamarg – Renowned for golden meadows and trekking.",
    "Yusmarg – Serene landscapes and pine forests."
  ]
}


In [ ]:
# Zero-shot prompting with LangChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,\
 HumanMessagePromptTemplate

system_template = """You are a sentiment analysis assistant.
                  Classify text as Positive, Negative, or Neutral."""

user_template = """Classify the following sentence:

                  {text}"""

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template)
])

llm = ChatOpenAI(model_name="gpt-4", temperature=0, api_key=openai.api_key)
chain = chat_prompt |llm

text_input = "The product is good, i dont mind buying again."

result = chain.invoke({"text": text_input})
print("Classification:", result.content)

Classification: Positive


In [ ]:
# Few shot prompting with LangChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,\
 HumanMessagePromptTemplate

examples = """Text: The service was very slow.
              Rating: 1

              Text: The steak was awfully good!.
              Rating: 5

              Text: It was ok, no massive compliants.
              Rating: 3"""

system_template = """You are a helpful assistant that classifies
                      text into ratings between 1 and 5 with
                      lowest is 1 and highest is 5"""


user_template = examples+ """

                Text:{text}
                Rating:
                """

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template)
])

llm = ChatOpenAI(model_name="gpt-4.1", temperature=0.3,
                 model_kwargs={"top_p": 0.4},
                 api_key=openai.api_key)

chain = chat_prompt | llm

text_input = "Meal was decent, but I've had better"

result = chain.invoke({"text": text_input})

print(result.content)


Rating: 3


In [ ]:
# Multi-step prompting with LangChain

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,\
 HumanMessagePromptTemplate

system_template = """You are a blog writer who writes in a pragmatic style
                      with no more than 100 words. Output your key summary in
                      no more than 5 bullet points """

user_template = """Compose a travel blog on {text} as follows:
Step 1: Introduce the destination
Step 2: Share personal adventures during the trip
Step 3: Summarize the journey """

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template)
])

llm = ChatOpenAI(model_name="gpt-4.1", temperature=0.3,
                 model_kwargs={"top_p": 0.4},
                 api_key=openai.api_key)

chain = chat_prompt | llm

text_input = "Goa"

result = chain.invoke({"text": text_input})

print(result.content)



Goa, India’s beach paradise, is a vibrant blend of sun, sand, and culture. From lively markets to serene churches, it’s a destination that caters to every traveler.

During my trip, I soaked up the sun at Palolem Beach, explored the historic forts of Aguada and Chapora, and sampled spicy Goan curries at a beach shack. A sunset cruise on the Mandovi River was a highlight, offering stunning views and local music.

Goa’s laid-back vibe, friendly locals, and diverse experiences make it a must-visit. I left with unforgettable memories and a longing to return.

**Key Summary:**
- Goa offers beautiful beaches and rich culture.
- Explored forts, beaches, and local cuisine.
- Enjoyed a sunset cruise on the Mandovi River.
- Friendly locals and relaxed atmosphere.
- Left with lasting memories and a desire to revisit.


In [ ]:
# Chain-of-Thought
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,\
 HumanMessagePromptTemplate

system_template = """You are a mathematics arithmetics specialist.
                      Your objective is to solve problems """

user_template = """ {text}
A: Let's think step by step
"""

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template)
])

llm = ChatOpenAI(model_name="gpt-4.1", temperature=0.3,
                 model_kwargs={"top_p": 0.4},
                 api_key=openai.api_key)

chain = chat_prompt | llm

text_input = """Q: You start with 15 books in your collection. At the bookstore,
you purchase 8 new books. Then, you lend 3 to your friend and 2 to your cousin.
Later, you visit another bookstore and buy 5 more books. How many books do you
have now? """

result = chain.invoke({"text": text_input})

print(result.content)


Sure! Let's solve the problem step by step:

1. **Start with 15 books.**
2. **Purchase 8 new books:**  
   15 + 8 = 23 books
3. **Lend 3 to your friend:**  
   23 - 3 = 20 books
4. **Lend 2 to your cousin:**  
   20 - 2 = 18 books
5. **Buy 5 more books:**  
   18 + 5 = 23 books

**Final Answer:**  
You have **23 books** now.


In [ ]:
# Self-consistency
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,\
 HumanMessagePromptTemplate

system_template = """You are a mathematics arithmetics specialist.
                      Your objective is to solve problems """

self_consistency_instruction = """Imagine three completely independent experts
who reason differently are answering the question. The final answer is obtained
by majority vote."""

user_template = self_consistency_instruction + """\n The question is: {text}"""

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template)
])

llm = ChatOpenAI(model_name="gpt-4.1", temperature=0.2,
                 model_kwargs={"top_p": 0.3},
                 api_key=openai.api_key)

chain = chat_prompt | llm

text_input = """If there are 10 cars in the parking lot and 3 more cars arrive.
Half the original number of cars leave. Then, half of the current number of cars
arrive. How many cars are there in the parking?"""

result = chain.invoke({"text": text_input})

print(result.content)




Let’s have three independent experts reason through the problem:

---

**Expert 1: Step-by-step calculation**

1. **Start:** 10 cars in the parking lot.
2. **3 more arrive:** 10 + 3 = 13 cars.
3. **Half the original number (10) leave:** 10 / 2 = 5 leave. 13 - 5 = 8 cars remain.
4. **Half of the current number (8) arrive:** 8 / 2 = 4 arrive. 8 + 4 = **12 cars**.

---

**Expert 2: Logical breakdown**

- Initial: 10 cars.
- 3 arrive: 13.
- 5 (half of 10) leave: 13 - 5 = 8.
- 4 (half of 8) arrive: 8 + 4 = **12 cars**.

---

**Expert 3: Algebraic approach**

Let’s use variables:
- Start: \( x = 10 \)
- After 3 arrive: \( x + 3 = 13 \)
- After half original leave: \( 13 - (10/2) = 13 - 5 = 8 \)
- After half current arrive: \( 8 + (8/2) = 8 + 4 = 12 \)

---

**Majority vote:**  
All three experts agree.

**Final answer:**  
\[
\boxed{12}
\]  
There are **12 cars** in the parking lot.


In [ ]:
# Tree-Of-Thought
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

# Solutions Chain
solutions_template = """
Generate {num_solutions} distinct solutions for {problem}. Consider
factors like {factors}.

Solutions:
"""

solutions_prompt = PromptTemplate(
    template=solutions_template,
    input_variables=["problem","factors","num_solutions"]
    )

llm_1 = ChatOpenAI(model_name="gpt-4.1", temperature=0.2,
                 model_kwargs={"top_p": 0.3},
                 api_key=openai.api_key)

solutions_chain = LLMChain(llm=llm_1,
                           prompt=solutions_prompt,
                           output_key="solutions")

# Evaluation Chain
evaluation_template = """
Evaluate each solution in {solutions} by analyzing pros, cons, feasibility, and
probability of success.

Evaluations:
"""

evaluation_prompt = PromptTemplate(
    template=evaluation_template,
    input_variables=["solutions"]
)

llm_2 = ChatOpenAI(model_name="gpt-4.1", temperature=0.2,
                 model_kwargs={"top_p": 0.3},
                 api_key=openai.api_key)

evaluation_chain = LLMChain(llm=llm_2,
                            prompt=evaluation_prompt,
                            output_key="evaluations")


# Reasoning Chain
reasoning_template = """
For the most promising solutions in {evaluations}, explain scenarios,
implementation strategies, partneships needed, and handling potential obstacles.

Enhanced Resoning:
"""

reasoning_prompt = PromptTemplate(
    template=reasoning_template,
    input_variables=["evaluations"]
)

llm_3 = ChatOpenAI(model_name="gpt-4.1", temperature=0.2,
                 model_kwargs={"top_p": 0.3},
                 api_key=openai.api_key)

reasoning_chain = LLMChain(llm= llm_3,
                           prompt= reasoning_prompt,
                           output_key="enhanced_reasoning")

#Ranking Chain

ranking_template = """
Based on the evaluations and reasoning, rank the solutions in
{enhanced_reasoning} from most to least promising.

Ranked Solutions:
"""
ranking_prompt = PromptTemplate(
    template=ranking_template,
    input_variables=["enhanced_reasoning"]
)

llm_4 = ChatOpenAI(model_name="gpt-4.1", temperature=0.2,
                 model_kwargs={"top_p": 0.3},
                 api_key=openai.api_key)


ranking_chain = LLMChain(llm= llm_4,
                           prompt= ranking_prompt,
                           output_key="ranked_solutions")

# Adding all chains in sequence
from langchain.chains import SequentialChain

total_chain = SequentialChain(
    chains = [solutions_chain, evaluation_chain, reasoning_chain,
              ranking_chain],
    input_variables = ["problem","factors","num_solutions"],
    output_variables= ["ranked_solutions"]
)

print(total_chain.run(
    problem = "Purchase Car in Bangalore",
    factors = """Requirements for high power, low maintenance, and moderate
    mileage""",
    num_solutions=3
))


Certainly! Based on the detailed evaluations, scenarios, implementation strategies, partnerships, and ways to handle obstacles, here is the **ranked list of solutions from most to least promising for a car purchase in Bangalore**:

---

## **1. Hyundai Creta 1.5L Petrol (Compact SUV)**
**Most Promising**

- **Why?**  
  - Best suited for Bangalore’s mixed and often poor road conditions due to higher ground clearance and robust build.
  - Offers modern features, safety tech, and comfort for families.
  - Versatile for both city and highway use.
  - High resale value and strong after-sales network.
- **Obstacles Handled:**  
  - Early booking and dealer negotiation can mitigate long waiting periods.
  - Parking aids and driving practice address size/parking concerns.
  - Maintenance packages help with running costs.

---

## **2. Honda City 1.5L VTEC (Petrol Sedan)**
**Very Promising**

- **Why?**  
  - Excellent for those prioritizing comfort, driving pleasure, and long-term reliability